In [1]:
from marvin_python_toolbox.common.data import MarvinData
import pandas as pd

initial_dataset = pd.read_csv(MarvinData.download_file("https://s3.amazonaws.com/automl-example/produtos.csv"), delimiter=";")

initial_dataset["text"] = initial_dataset["nome"] + " " + initial_dataset["descricao"]
initial_dataset.drop(["descricao", "nome"], axis=1, inplace=True)
initial_dataset.dropna(inplace=True)

print(initial_dataset.head())

  categoria                                               text
0     livro   O Hobbit - 7ª Ed. 2013  Produto NovoBilbo Bol...
1     livro   Livro - It A Coisa - Stephen King  Produto No...
2     livro   Box  As Crônicas De Gelo E Fogo  Pocket  5 Li...
3     livro   Box Harry Potter  Produto Novo e Físico  A sé...
4     livro   Livro Origem - Dan Brown  Produto NovoDe Onde...


In [2]:
initial_dataset.head()

,categoria,text
0,livro,O Hobbit - 7ª Ed. 2013 Produto NovoBilbo Bol...
1,livro,Livro - It A Coisa - Stephen King Produto No...
2,livro,Box As Crônicas De Gelo E Fogo Pocket 5 Li...
3,livro,Box Harry Potter Produto Novo e Físico A sé...
4,livro,Livro Origem - Dan Brown Produto NovoDe Onde...


In [3]:
marvin_initial_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2916 entries, 0 to 4079
Data columns (total 2 columns):
categoria    2916 non-null object
text         2916 non-null object
dtypes: object(2)
memory usage: 68.3+ KB


In [4]:
marvin_initial_dataset.categoria.value_counts()

livro        838
maquiagem    788
brinquedo    668
game         622
Name: categoria, dtype: int64

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    marvin_initial_dataset["text"], 
    marvin_initial_dataset["categoria"], 
    test_size = 0.2, 
    random_state = 10
)

vect = CountVectorizer()
vect.fit(marvin_initial_dataset["text"])

marvin_dataset = {
    "X_train": vect.transform(X_train),
    "X_test": vect.transform(X_test),
    "y_train": y_train,
    "y_test" : y_test,
    "vect": vect
}

In [6]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(marvin_dataset["X_train"], marvin_dataset["y_train"])

marvin_model = {
    "clf" : clf,
    "vect": marvin_dataset["vect"]
}

print("Done!!!")

Done!!!


In [7]:
from sklearn.metrics import accuracy_score

y_prediction = marvin_model["clf"].predict(marvin_dataset["X_test"])

accuracy = accuracy_score(y_prediction, marvin_dataset["y_test"])

print accuracy

0.979452054795


In [8]:
input_message = {
    "nome": "As Brumas De Avalon - o Clássico Que Encantou Gerações - Edição Especial - Capa Dura"
}

In [9]:
print input_message

input_message = marvin_model["vect"].transform([input_message["nome"]])

{'nome': 'As Brumas De Avalon - o Cl\xc3\xa1ssico Que Encantou Gera\xc3\xa7\xc3\xb5es - Edi\xc3\xa7\xc3\xa3o Especial - Capa Dura'}


In [10]:
final_prediction = marvin_model["clf"].predict(input_message)[0]